In [1]:
import yaml
import sys
import os 
import io
import numpy as np
import torch as th 
import wandb
import sys
import pandas as pd

from sklearn.model_selection import KFold
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader 

from tqdm import tqdm
import random 
from imblearn.over_sampling import SMOTE #, ADASYN

from src.data.data_loader import load_data, load_or_cache_data
from src.utility.ut_GNNbranch import relabel_train_outcome, recover_original_label
from src.utility.ut_general import name_to_model, dict_to_namespace, namespace_to_dict



In [5]:

rootfolder = "C:\\Users\\tosso\\Documents\\widsdatathon2025"
sys.path.append(os.path.join(rootfolder))
#datafolder = os.path.join(rootfolder, "data")
datafolder = rootfolder

pickle_file = os.path.join(datafolder, "data.pkl") 
train_data_dic, test_data_dic = load_or_cache_data(datafolder, pickle_file)

Loading data from cached files -> Done!


In [6]:
train_data_dic.update(test_data_dic)

In [4]:
# Alabama Parenting Questionnaire: 
# Strengths and Difficulties: https://acamh.onlinelibrary.wiley.com/doi/epdf/10.1111/j.1469-7610.1997.tb01545.x
# To be consistent with connectome, normalize the values to be between 0 and 1
normalizing_factors = {
    "APQ_P_APQ_P_CP": 3 * 5,
    "APQ_P_APQ_P_PP": 6 * 5,
    "APQ_P_APQ_P_PM": 10 * 5,
    "APQ_P_APQ_P_OPD": 7 * 5,
    "APQ_P_APQ_P_INV": 10 * 5,
    "APQ_P_APQ_P_ID": 6 * 5,
    "SDQ_SDQ_Hyperactivity": 10.0,
    "SDQ_SDQ_Peer_Problems": 10.0,
    "SDQ_SDQ_Conduct_Problems": 10.0,
    "SDQ_SDQ_Emotional_Problems": 10.0,
    "SDQ_SDQ_Difficulties_Total": 40.0,
    "SDQ_SDQ_Prosocial": 10.0,
    "SDQ_SDQ_Externalizing": 20.0, # Sum of conduct + hyperactivity (https://www.sdqinfo.org/a0.html)
    "SDQ_SDQ_Generating_Impact": 10.0,
    "SDQ_SDQ_Internalizing": 20.0, # Sum of emotional + peer symptoms (https://www.sdqinfo.org/a0.html)
}



In [9]:
from src.data.KNN_imputer import KNNImputer_with_OneHotEncoding

imputer = KNNImputer_with_OneHotEncoding()
metadata = imputer.fit_transform(train_data_dic, split="test")

def encode_column_into_bins(df, column, bins, labels):
    df['binned'] = pd.cut(df[column], bins=bins, labels=labels, right=True)
    df_encoded = pd.get_dummies(df, columns=['binned'], prefix='', prefix_sep='', dtype=float)
    df_encoded = df_encoded.drop(columns=[column])
    return df_encoded

# Special case handedness column
metadata['EHQ_EHQ_Total'] = metadata['EHQ_EHQ_Total'] / 200 + 0.5
metadata = encode_column_into_bins(metadata, 'ColorVision_CV_Score' , [0, 12, 100], ['Color_Blind', 'Normal_Vision'])
metadata = encode_column_into_bins(metadata, 'MRI_Track_Age_at_Scan' , [0, 4, 11, 17, 30], ['Infant', 'Child', "Adolescent", "Adult"])

# Normalize everything to be between 0 and 1
for col in normalizing_factors:
    metadata[col] /= normalizing_factors[col]

# Remove features in train which never appear
columns_which_dont_appear_in_train = ["Basic_Demos_Study_Site_5", "PreInt_Demos_Fam_Child_Race_-1", "Barratt_Barratt_P1_Edu_-1", "Barratt_Barratt_P1_Occ_-1", "Barratt_Barratt_P2_Edu_-1", "Barratt_Barratt_P2_Occ_-1", "Infant"]
for col in columns_which_dont_appear_in_train:
    metadata = metadata.drop(columns=[col])
        
metadata


,EHQ_EHQ_Total,APQ_P_APQ_P_CP,APQ_P_APQ_P_ID,APQ_P_APQ_P_INV,APQ_P_APQ_P_OPD,APQ_P_APQ_P_PM,APQ_P_APQ_P_PP,SDQ_SDQ_Conduct_Problems,SDQ_SDQ_Difficulties_Total,SDQ_SDQ_Emotional_Problems,...,Barratt_Barratt_P2_Occ_30,Barratt_Barratt_P2_Occ_35,Barratt_Barratt_P2_Occ_40,Barratt_Barratt_P2_Occ_45,Barratt_Barratt_P2_Occ_5,Color_Blind,Normal_Vision,Child,Adolescent,Adult
participant_id,,,,,,,,,,,,,,,,,,,,,
Cfwaf5FX7jWK,0.80015,0.333333,0.533333,0.82,0.542857,0.22,0.866667,0.2,0.300,0.3,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
vhGrzmvA3Hjq,0.93355,0.200000,0.433333,0.86,0.514286,0.30,0.933333,0.2,0.400,0.8,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
ULliyEXjy4OV,0.63340,0.200000,0.466667,0.72,0.457143,0.28,0.833333,0.1,0.175,0.1,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
LZfeAb1xMtql,0.96690,0.200000,0.633333,0.82,0.485714,0.36,0.900000,0.4,0.375,0.4,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
EnFOUv0YK1RG,0.03310,0.200000,0.433333,0.84,0.542857,0.32,0.933333,0.2,0.450,0.6,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
UadZfjdEg7eG,0.93355,0.200000,0.566667,0.82,0.485714,0.22,0.833333,0.1,0.450,0.7,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
IUEHiLmQAqCi,0.86685,0.333333,0.400000,0.76,0.628571,0.18,0.966667,0.2,0.400,0.2,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
cRySmCadYFRO,0.93920,0.200000,0.466667,0.84,0.428571,0.20,0.933333,0.1,0.275,0.4,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0


In [22]:
metadata.index.tolist().index('CPaeQkhcjg7d')

1

In [19]:
metadata.index[metadata['participant_id'] == 'CPaeQkhcjg7d']

KeyError: 'participant_id'

In [15]:
metadata.values

array([[0.7       , 0.2       , 0.33333333, ..., 1.        , 0.        ,
        0.        ],
       [0.02765   , 0.2       , 0.43333333, ..., 0.        , 1.        ,
        0.        ],
       [0.26665   , 0.26666667, 0.33333333, ..., 1.        , 0.        ,
        0.        ],
       ...,
       [0.5834    , 0.2       , 0.53333333, ..., 0.        , 1.        ,
        0.        ],
       [0.767     , 0.2       , 0.46666667, ..., 0.        , 1.        ,
        0.        ],
       [0.211     , 0.33333333, 0.36666667, ..., 0.        , 1.        ,
        0.        ]], shape=(1213, 81))

In [6]:
from matplotlib import pyplot as plt

for col in metadata.columns:
    print(col, max(metadata.loc[:, col]))

EHQ_EHQ_Total 1.0
APQ_P_APQ_P_CP 0.8
APQ_P_APQ_P_ID 0.9333333333333333
APQ_P_APQ_P_INV 1.0
APQ_P_APQ_P_OPD 0.8
APQ_P_APQ_P_PM 0.74
APQ_P_APQ_P_PP 1.0
SDQ_SDQ_Conduct_Problems 1.0
SDQ_SDQ_Difficulties_Total 0.85
SDQ_SDQ_Emotional_Problems 1.0
SDQ_SDQ_Externalizing 1.0
SDQ_SDQ_Generating_Impact 1.0
SDQ_SDQ_Hyperactivity 1.0
SDQ_SDQ_Internalizing 0.85
SDQ_SDQ_Peer_Problems 0.9
SDQ_SDQ_Prosocial 1.0
Basic_Demos_Study_Site_1 1.0
Basic_Demos_Study_Site_2 1.0
Basic_Demos_Study_Site_3 1.0
Basic_Demos_Study_Site_4 1.0
Basic_Demos_Study_Site_5 0.0
PreInt_Demos_Fam_Child_Ethnicity_-1 1.0
PreInt_Demos_Fam_Child_Ethnicity_0 1.0
PreInt_Demos_Fam_Child_Ethnicity_1 1.0
PreInt_Demos_Fam_Child_Ethnicity_2 1.0
PreInt_Demos_Fam_Child_Ethnicity_3 1.0
PreInt_Demos_Fam_Child_Race_-1 0.0
PreInt_Demos_Fam_Child_Race_0 1.0
PreInt_Demos_Fam_Child_Race_1 1.0
PreInt_Demos_Fam_Child_Race_10 1.0
PreInt_Demos_Fam_Child_Race_11 1.0
PreInt_Demos_Fam_Child_Race_2 1.0
PreInt_Demos_Fam_Child_Race_3 1.0
PreInt_Demos_Fam_Ch

In [12]:
train_data_dic.update(test_data_dic)

In [13]:
len(train_data_dic)

7

In [14]:
print(len(train_data_dic['train_cate']), len(train_data_dic['train_quant']))

1213 1213


In [15]:
train_data_dic['train_quant']

,participant_id,EHQ_EHQ_Total,ColorVision_CV_Score,APQ_P_APQ_P_CP,APQ_P_APQ_P_ID,APQ_P_APQ_P_INV,APQ_P_APQ_P_OPD,APQ_P_APQ_P_PM,APQ_P_APQ_P_PP,SDQ_SDQ_Conduct_Problems,SDQ_SDQ_Difficulties_Total,SDQ_SDQ_Emotional_Problems,SDQ_SDQ_Externalizing,SDQ_SDQ_Generating_Impact,SDQ_SDQ_Hyperactivity,SDQ_SDQ_Internalizing,SDQ_SDQ_Peer_Problems,SDQ_SDQ_Prosocial,MRI_Track_Age_at_Scan
0,UmrK0vMLopoR,40.00,13,3,10,47,13,11,28,0,6,1,5,0,5,1,0,10,NaN
1,CPaeQkhcjg7d,-94.47,14,3,13,34,18,23,30,0,18,6,8,7,8,10,4,5,NaN
2,Nb4EetVPm3gs,-46.67,14,4,10,35,16,10,29,1,14,2,8,5,7,6,4,9,8.239904
3,p4vPhVu91o4b,-26.68,10,5,12,39,19,16,28,6,24,4,16,9,10,8,4,6,NaN
4,M09PXs7arQ5E,0.00,14,5,15,40,20,24,28,1,18,4,11,4,10,7,3,9,8.940679
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1208,Atx7oub96GXS,87.80,14,5,14,39,20,15,21,1,9,2,7,3,6,2,0,9,10.697923
1209,groSbUfkQngM,77.80,14,3,10,32,20,11,23,6,18,3,11,9,5,7,4,7,13.964750
1210,zmxGvIrOD0bt,16.68,14,3,16,28,15,19,27,3,4,1,3,0,0,1,0,10,NaN
1211,rOmWFuJCud5G,53.40,14,3,14,34,18,23,23,4,9,0,9,3,5,0,0,9,12.089094


In [20]:
# Merge on participant_ID
df = pd.merge(train_data_dic['train_cate'], train_data_dic['train_quant'], on='participant_id', how='inner')  # Options: 'inner', 'outer', 'left', 'right'

In [17]:
df = train_data_dic['train_cate']
#pd.get_dummies(df, columns=df.columns[1:])
df

,participant_id,Basic_Demos_Enroll_Year,Basic_Demos_Study_Site,PreInt_Demos_Fam_Child_Ethnicity,PreInt_Demos_Fam_Child_Race,MRI_Track_Scan_Location,Barratt_Barratt_P1_Edu,Barratt_Barratt_P1_Occ,Barratt_Barratt_P2_Edu,Barratt_Barratt_P2_Occ
0,UmrK0vMLopoR,2016,1,0.0,0,1,21,45,21,45
1,CPaeQkhcjg7d,2019,3,1.0,2,3,15,15,0,0
2,Nb4EetVPm3gs,2016,1,1.0,8,1,18,40,0,0
3,p4vPhVu91o4b,2018,3,0.0,8,3,15,30,18,0
4,M09PXs7arQ5E,2019,3,0.0,1,3,15,20,0,0
...,...,...,...,...,...,...,...,...,...,...
1208,Atx7oub96GXS,2019,1,0.0,0,2,21,40,21,40
1209,groSbUfkQngM,2016,1,0.0,0,1,18,35,0,0
1210,zmxGvIrOD0bt,2015,1,1.0,2,1,18,20,0,0
1211,rOmWFuJCud5G,2018,3,0.0,8,3,21,40,18,45


In [23]:
# Survey Info: https://www.youthcoalition.net/wp-content/uploads/2022/06/APQ.pdf 
templates = {
    "Basic_Demos_Enroll_Year": {
        "prompt": "The scan was taken in {}.",
        "mapping": {}
    },
    "Basic_Demos_Study_Site": {
        "prompt": "The site of phenotypic testing was {}.",
        "mapping": {
            "1": "Staten Island",
            "2": "MRV",
            "3": "Midtown",
            "4": "Harlem",
            "5": "SI RUMC"
        }
    },
     "MRI_Track_Scan_Location": {
        "prompt": "The site of the MRI scan was {}.",
        "mapping": {
            "1": "Staten Island",
            "2": "RUBIC",
            "3": "CBIC",
            "4": "CUNY"
        }
    },
    "PreInt_Demos_Fam_Child_Ethnicity": {
        "prompt": "The patient's ethnicity is {}.",
        "mapping": {
            "0": "not Hispanic or Latino",
            "1": "Hispanic or Latino",
            "2": "unknown because the patient declined to specify",
            "3": "unknown"
        }
    },
    "PreInt_Demos_Fam_Child_Race": {
        "prompt": "The patient's race is {}.",
        "mapping": {
            "0": "White/Caucasian",
            "1": "Black/African American",
            "2": "Hispanic",
            "3": "Asian",
            "4": "Indian",
            "5": "Native American Indian",
            "6": "American Indian/Alaskan Native",
            "7": "Native Hawaiian/Other Pacific Islander",
            "8": "Two or more races",
            "9": "Other race",
            "10": "Unknown",
            "11": "Unknown because chose not to specify"
        }
    },
    "Barratt_Barratt_P1_Edu": {
        "prompt": "The patient's first parent {}.",
        "mapping": {
            "0": "has no formal education",
            "3": "has less than 7th grade education",
            "6": "attended junior high/middle school (8th grade)",
            "9": "completed partial high school (10th or 11th grade)",
            "12": "graduated high school",
            "15": "completed at least one year of college",
            "18": "has a college education",
            "21": "earned a graduate degree"
        }
    },
    "Barratt_Barratt_P1_Occ": {
        "prompt": "The patient's first parent's occupation is one of {}.",
        "mapping": {
            "0": "homemaker, stay at home parent",
            "5": "day laborer, janitor, house cleaner, farm worker, food counter sales, food preparation worker, busboy",
            "10": "garbage collector, short-order cook, cab driver, shoe sales, assembly line worker, mason, baggage porter",
            "15": "painter, skilled construction trade, sales clerk, truck driver, cook, sales counter or general office clerk",
            "20": "automobile mechanic, typist, locksmith, farmer, carpenter, receptionist, construction laborer, hairdresser",
            "25": "machinist, musician, bookkeeper, secretary, insurance sales, cabinet maker, personnel specialist, welder",
            "30": "supervisor, librarian, aircraft mechanic, artist and artisan, electrician, administrator, military enlisted personnel, buyer",
            "35": "nurse, skilled technician, medical technician, counselor, manager, police and fire personnel, financial manager, physical, occupational, speech therapist",
            "40": "mechanical, nuclear, and electrical engineer, educational administrator, veterinarian, military officer, elementary, high school, and special education teacher",
            "45": "physician, attorney, professor, chemical and aerospace engineer, judge, CEO, senior manager, public official, psychologist, pharmacist, accountant"
        }
    },
    "Barratt_Barratt_P2_Edu": {
        "prompt": "The patient's second parent {}.",
        "mapping": {
            "0": "has no formal education",
            "3": "has less than 7th grade education",
            "6": "attended junior high/middle school (8th grade)",
            "9": "completed partial high school (10th or 11th grade)",
            "12": "graduated high school",
            "15": "completed at least one year of college",
            "18": "has a college education",
            "21": "earned a graduate degree"
        }
    },
    "Barratt_Barratt_P2_Occ": {
        "prompt": "The patient's second parent's occupation is one of {}.",
        "mapping": {
            "0": "homemaker, stay at home parent",
            "5": "day laborer, janitor, house cleaner, farm worker, food counter sales, food preparation worker, busboy",
            "10": "garbage collector, short-order cook, cab driver, shoe sales, assembly line worker, mason, baggage porter",
            "15": "painter, skilled construction trade, sales clerk, truck driver, cook, sales counter or general office clerk",
            "20": "automobile mechanic, typist, locksmith, farmer, carpenter, receptionist, construction laborer, hairdresser",
            "25": "machinist, musician, bookkeeper, secretary, insurance sales, cabinet maker, personnel specialist, welder",
            "30": "supervisor, librarian, aircraft mechanic, artist and artisan, electrician, administrator, military enlisted personnel, buyer",
            "35": "nurse, skilled technician, medical technician, counselor, manager, police and fire personnel, financial manager, physical, occupational, speech therapist",
            "40": "mechanical, nuclear, and electrical engineer, educational administrator, veterinarian, military officer, elementary, high school, and special education teacher",
            "45": "physician, attorney, professor, chemical and aerospace engineer, judge, CEO, senior manager, public official, psychologist, pharmacist, accountant"
        }
    },
    "APQ_P_APQ_P_CP": {
        "prompt": "Corporal Punishment: The parents **{}** use spanking and other forms of physical punishment as a response to misbehavior.",
        "mapping": {
            "1": "never",
            "2": "almost never",
            "3": "sometimes",
            "4": "often",
            "5": "always"
        }
    },
    "APQ_P_APQ_P_ID": {
        "prompt": "Inconsistency: The parents **{}** are inconsistent in enforcing rules/consequences.",
        "mapping": {
            "1": "never",
            "2": "almost never",
            "3": "sometimes",
            "4": "often",
            "5": "always"
        }
    },
    "APQ_P_APQ_P_INV": {
        "prompt": "Involvement: The parents are **{}** involved in helping with homework, spending quality time, and showing interest in the child's activities",
        "mapping": {
            "1": "never",
            "2": "almost never",
            "3": "sometimes",
            "4": "often",
            "5": "always"
        }
    },
    "APQ_P_APQ_P_OPD": {
        "prompt": "The parents **{}** use non-physical discipline strategies, such as time-outs or removal of privileges.",
        "mapping": {
            "1": "never",
            "2": "almost never",
            "3": "sometimes",
            "4": "often",
            "5": "always"
        }
    },
    "APQ_P_APQ_P_PM": {
        "prompt": "The parents **{}** fail to monitor their child, including not knowing their child's whereabouts, activities, or peers.",
        "mapping": {
            "1": "never",
            "2": "almost never",
            "3": "sometimes",
            "4": "often",
            "5": "always"
        }
    },
    "APQ_P_APQ_P_PP": {
        "prompt": "The parents **{}** use positive reinforcement and encouragement, such as praising the child for good behavior and providing warmth and affection.",
        "mapping": {
            "1": "never",
            "2": "almost never",
            "3": "sometimes",
            "4": "often",
            "5": "always"
        }
    }
}


"""The items on the adult form are categorized into five subscales as follows:
o Involvement: 1, 4, 7, 9, 11, 14, 15, 20, 23, 26
o Positive Parenting: 2, 5, 13, 16, 18, 27
o Poor Monitoring/Supervision: 6, 10, 17, 19, 21, 24, 28, 29, 30, 32
o Inconsistent Discipline: 3, 8, 12, 22, 25, 31
o Corporal Punishment: 33, 35, 39
"""

APQ_question_counts = {
    "APQ_P_APQ_P_CP": 3,
    "APQ_P_APQ_P_PP": 6,
    "APQ_P_APQ_P_PM": 10,
    "APQ_P_APQ_P_OPD": 7,
    "APQ_P_APQ_P_INV": 10,
    "APQ_P_APQ_P_ID": 6
}

APQ_question_counts = {
    "APQ_P_APQ_P_CP": 3,
    "APQ_P_APQ_P_PP": 6,
    "APQ_P_APQ_P_PM": 10,
    "APQ_P_APQ_P_OPD": 7,
    "APQ_P_APQ_P_INV": 10,
    "APQ_P_APQ_P_ID": 6
}

def get_patient_report(row):
    report = "**Experiment Background:**\n\n"
    for col in ["Basic_Demos_Enroll_Year", "Basic_Demos_Study_Site", "MRI_Track_Scan_Location"]:
        val = str(int(row[col]))
        report += templates[col]["prompt"].format(templates[col]["mapping"].get(val, val)) + "\n"
    report = report[:-1]
    report += "\n\n**Demographic Information:**\n\n"
    for col in ["PreInt_Demos_Fam_Child_Ethnicity", "PreInt_Demos_Fam_Child_Race", "Barratt_Barratt_P1_Edu", "Barratt_Barratt_P1_Occ", "Barratt_Barratt_P2_Edu", "Barratt_Barratt_P2_Occ"]:
        val = str(int(row[col]))
        report += templates[col]["prompt"].format(templates[col]["mapping"].get(val, val)) + "\n"
    report = report[:-1]
    report += "\n\n**Parenting Style:**\n\n"
    for col in ["APQ_P_APQ_P_INV", "APQ_P_APQ_P_ID", "APQ_P_APQ_P_PM", "APQ_P_APQ_P_CP", "APQ_P_APQ_P_PP", "APQ_P_APQ_P_OPD"]:
        val = str(round(int(row[col]) / APQ_question_counts[col]))
        report += templates[col]["prompt"].format(templates[col]["mapping"].get(val, val)) + "\n"
    report = report[:-1]
    report += "\n\n**Summary of Strengths and Difficulties:**\n\n"
    for col in ["APQ_P_APQ_P_INV", "APQ_P_APQ_P_ID", "APQ_P_APQ_P_PM", "APQ_P_APQ_P_CP", "APQ_P_APQ_P_PP", "APQ_P_APQ_P_OPD"]:
        val = str(round(int(row[col]) / APQ_question_counts[col]))
        report += templates[col]["prompt"].format(templates[col]["mapping"].get(val, val)) + "\n"
    report = report[:-1]
    # report += "\n\n**Parenting Style:**\n\n"
    # for col in ["APQ_P_APQ_P_INV", "APQ_P_APQ_P_ID", "APQ_P_APQ_P_PM", "APQ_P_APQ_P_CP", "APQ_P_APQ_P_PP", "APQ_P_APQ_P_OPD"]:
    #     val = str(round(int(row[col]) / APQ_question_counts[col]))
    #     report += templates[col]["prompt"].format(templates[col]["mapping"].get(val, val)) + "\n"
    # report = report[:-1]
    return report

for i, row in df.iterrows():
    report = get_patient_report(row)
    print(report)
    break
    # if i == 3:
    #     break

**Experiment Background:**

The scan was taken in 2016.
The site of phenotypic testing was Staten Island.
The site of the MRI scan was Staten Island.

**Demographic Information:**

The patient's ethnicity is not Hispanic or Latino.
The patient's race is White/Caucasian.
The patient's first parent earned a graduate degree.
The patient's first parent's occupation is one of physician, attorney, professor, chemical and aerospace engineer, judge, CEO, senior manager, public official, psychologist, pharmacist, accountant.
The patient's second parent earned a graduate degree.
The patient's second parent's occupation is one of physician, attorney, professor, chemical and aerospace engineer, judge, CEO, senior manager, public official, psychologist, pharmacist, accountant.

**Parenting Style:**

Involvement: The parents are **always** involved in helping with homework, spending quality time, and showing interest in the child's activities
Inconsistency: The parents **almost never** are inconsist